In [175]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from datetime import datetime
from datetime import date
from copy import deepcopy
import json

# Import dataset
aurora_raw = pd.read_csv("customers (3).csv")

In [176]:
aurora_raw.head()
aurora_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Columns: 211 entries, @01_lactante to @zona_pais
dtypes: bool(2), float64(43), int64(2), object(164)
memory usage: 410.5+ KB


In [177]:
# Rename variables according to Carto names (first round)

newColumns = { 
                '@phone'	: 'phone',
                '@id'	: 'id',
                '@conexion_date' : 'conexion_date',
                '@date_registred' : 'date_registered',
                '@0_flow' : 'flow_type',
                '@1_gestor' : 'gestor',
                '@zona_pais' : 'zona_pais'
}

# there are still missing variables of dates, geographical coordinates, feeback of services.
aurora_raw = aurora_raw.rename(columns=newColumns)

In [178]:
#Eliminate observations of Aurora team phones. 
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56994154407 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56963085715 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56930150973 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56956195946 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573245502367 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573002983157 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573044640378 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573133910738 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573185891917 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573182396683 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573206646548 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573216903526 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 573105523263 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50685075883 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50660030436 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50763068014 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50768244084 ].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 51963852497 ].index)
print(aurora_raw.date_registered.info())


<class 'pandas.core.series.Series'>
Int64Index: 233 entries, 0 to 249
Series name: date_registered
Non-Null Count  Dtype 
--------------  ----- 
233 non-null    object
dtypes: object(1)
memory usage: 3.6+ KB
None


In [179]:
aurora_raw["date_registered"]= pd.to_datetime(aurora_raw["date_registered"], format='%d-%m-%Y %H:%M:%S' )
aurora_raw["date_"] = aurora_raw["date_registered"].dt.strftime("%Y-%m-%d")


In [180]:
aurora_raw["date_"]

0      2023-10-23
1      2023-10-23
2      2023-10-23
3      2023-10-23
4      2023-10-23
          ...    
245    2023-10-07
246    2023-10-04
247    2023-10-03
248    2023-10-03
249    2023-09-27
Name: date_, Length: 233, dtype: object

In [182]:


# totals by zona
unpivoted = aurora_raw.groupby(['zona_pais', 'date_'])['id'].count().reset_index()
pivoted = unpivoted.pivot(
   columns='date_',
   index= 'zona_pais',
  values='id')
# Reset the index of the pivoted DataFrame
pivoted = pivoted.reset_index()

# Filter rows in the pivoted DataFrame based on the 'flow' column in the original DataFrame
condition0 = aurora_raw['flow_type'] == "Enganche"
pivoted0 = pivoted[pivoted['zona_pais'].isin(aurora_raw[condition0]['zona_pais'])]

pivoted0



date_,zona_pais,2023-10-04,2023-10-11,2023-10-12,2023-10-13,2023-10-14,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-21,2023-10-23
0,Arica - Chile,1.0,3.0,NaN,7.0,3.0,5.0,1.0,12.0,NaN,10.0,NaN,NaN
1,Colchane - Bolivia,NaN,NaN,NaN,NaN,NaN,1.0,6.0,6.0,NaN,NaN,NaN,NaN
2,Colchane - Chile,NaN,NaN,NaN,NaN,NaN,1.0,5.0,5.0,7.0,4.0,NaN,NaN
3,Corredor - Costa Rica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,Cúcuta - Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN
5,Guadalupito - Perú,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
6,Ipiales - Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN,6.0
7,Iquique - Chile,NaN,9.0,7.0,NaN,NaN,5.0,2.0,NaN,NaN,2.0,NaN,1.0
9,Los patios - Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
10,Maturín - Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [187]:
# totals by organization
unpivoted = aurora_raw.groupby(['zona_pais', 'flow_type'])['id'].count().reset_index()
pivoted = unpivoted.pivot(
   columns='flow_type',
   index= 'zona_pais',
  values='id')
# Reset the index of the pivoted DataFrame
pivoted = pivoted.reset_index()


In [188]:
excel_file = 'enganches.xlsx'
pivoted.to_excel(excel_file, index=False)